In [ ]:
from addict import Dict
from pathlib import Path
from data import 
import numpy as np
import os

data_dir = os.environ["DATA_DIR"]
process_dir = data_dir / "processed"
args = Dict({
    "device": "cpu", # set to "cuda" if gpu is available
    "out_dir": data_dir / "predictions"
})

In [ ]:
import torch
from unet import Unet

state = torch.load(data_dir / "model.pt", map_location=args.device)
model = Unet(13, 3, 4).to(args.device)
model.load_state_dict(state)
model = model.eval()

In [ ]:
from data import GlacierDataset
from torch.utils.data import DataLoader

paths = {}
for split in ["train", "test"]:
    paths[split] = {}
    for v in ["x", "y"]:
        paths[split][v] = list((process_dir / split).glob(v + "*"))
        paths[split][v].sort()

ds = {
    "train": GlacierDataset(paths["train"]["x"], paths["train"]["y"]),
    "test": GlacierDataset(paths["test"]["x"], paths["test"]["y"])
}

In [ ]:
from data import predictions

predictions(model, ds["train"], args.out_dir / "train", args.device)
predictions(model, ds["test"], args.out_dir / "test", args.device)